In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import time

pd.options.mode.chained_assignment = None  # default='warn'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
## https://developers.refinitiv.com/article/find-your-right-companies-screener-eikon-data-apispython
# https://www.reddit.com/r/webdev/comments/t2s836/what_is_the_easiest_way_to_hide_your_api_key_on/
import eikon as ek
ek.set_app_key('341ab68907a740f38e7b07084d4055e3982b5bdf')

## M&A screening

<pre>•Deals zwischen Juni 2008 und September 2022
•Deal status: Completed & withdrawn
•Nation classification: US,Canada,Australia, United Kingdom
•Deal value: größer 1Mio USD
•M&A Type: Disclosed dollar value deal
•% of shares before deal kleiner 50%, danach größer 50%
•Company status: public</pre>

In [98]:
'''
unbedingt "&" removen aus der excel funktion. den rest kann man anscheinend direkt übernehmen
            BETWEEN(TR.MnAAnnDate,20210112,20211512), \
            BETWEEN(TR.MnAAnnDate,20080107,20213112), \
            IN(TR.MnAAcquirorPublicStatus,"P"), \ #also include subsidiaries (most of them are irrelevant though)
'''

firms = 'SCREEN(U(IN(DEALS)),' + ' \
            BETWEEN(TR.MnAAnnDate,20080107,20213112), \
            IN(TR.MnAAcquirorNation,"AU","GB","CA","US"), \
            IN(TR.MnATargetNation,"AU","GB","CA","US"), \
            IN(TR.MnAStatus,"W","SW","IW","DR","U","C"), \
            TR.MnADealValue(Scale=6)>=1, \
            IN(TR.MnADealType,"DI"), \
            TR.MnAPctOfSharesHeld6MonthsPrior<50, \
            TR.MnAPctOfSharesOwnedPostMerger>50, \
            IN(TR.MnAAcquirorPublicStatus,"P", "S"), \
            IN(TR.MnATargetPublicStatus,"P", "S"), \
            CURN=USD) \
            '
fields = ['TR.MnASDCDealNumber,TR.MnAAnnDate,TR.MnADateUnconditional,TR.MnAEffectiveDate,TR.MnAStatus,TR.MnATarget,TR.MnATargetPermId,TR.MnATargetSDCCusip,TR.MnATargetNation,TR.MnATargetPublicStatus,TR.MnATargetTotalAssetsLTM,TR.MnAAcquiror,TR.MnAAcquirorPermId,TR.MnAAcquirorSDCCusip,TR.MnAAcquirorSEDOL,TR.MnAAcquirorNation,TR.MnAAcquirorPublicStatus,TR.MnAAcquirorTotalAssetsLTM, TR.MnADealValue(Curn=USD,Scale=6),TR.MnAPctOfSharesHeld6MonthsPrior,TR.MnAPctOfSharesOwnedPostMerger'] #can be placed in just one string

fields_example = ['TR.MnASDCDealNumber', 'TR.MnAAnnDate', 'TR.MnATarget', 'TR.MnATargetPermId', 'TR.MnAAcquiror', 'TR.MnAAcquirorPermId']

df_ma, err = ek.get_data(firms, fields)
df_ma.head()
df_ma.shape

'\nunbedingt "&" removen aus der excel funktion. den rest kann man anscheinend direkt übernehmen\n            BETWEEN(TR.MnAAnnDate,20210112,20211512),             BETWEEN(TR.MnAAnnDate,20080107,20213112),             IN(TR.MnAAcquirorPublicStatus,"P"), \\ #also include subsidiaries (most of them are irrelevant though)\n'

,Instrument,SDC Deal No,Date Announced,Date Effective or Unconditional,Date Effective,Deal Status,Target Full Name,Target PermID,Target 6-digit CUSIP,Target Nation,...,Acquiror Full Name,Acquiror PermID,Acquiror 6-digit CUSIP,Acquiror's SEDOL,Acquiror Nation,Acquiror Public Status,Acquiror Total Assets Last 12 Months,Deal Value,Percentage of Shares Held by Acquiror 6 Months Prior to Announcement,Percentage of Shares Owned after Transaction
0,34391097466,2116013020,2009-10-07,2009-12-08,2009-12-08,Completed,Trinity Alps Resources Inc- Discovery Day Gold...,<NA>,25581K,United States,...,American Sierra Gold Corp,4298488815.0,029616,B2NS4Z7,United States,Public,<NA>,3.7,0.0,75.0
1,34391276852,1761253020,2011-03-28,2011-05-02,2011-05-02,Completed,Fulton Street Brewery LLC,5035166382.0,14681L,United States,...,Anheuser Busch Inc,4296453080.0,035231,<NA>,United States,Subsidiary,<NA>,22.5,0.0,58.0
2,34391279156,1780964020,2010-05-21,2010-06-29,2010-06-29,Completed,Michael Foods Inc,4295907158.0,594079,United States,...,GS Capital Partners LP,5000878558.0,36224Z,<NA>,United States,Subsidiary,<NA>,1700.0,0.0,100.0
3,34391281082,1794406020,2008-09-15,2008-10-30,2008-10-30,Completed,Napster Inc,4295899555.0,630797,United States,...,Best Buy Co Inc,4295903556.0,086516,2094670,United States,Public,17522000000.0,126.935,0.0,90.191
4,34391290134,1879131020,2008-04-24,2008-09-29,2008-09-29,Completed,Wendy's International Inc,4295905330.0,950590,United States,...,Triarc Cos Inc,4295905130.0,895927,B3NXMJ9,United States,Public,1390735000.0,2345.517,0.0,100.0


(15575, 22)

In [99]:
## save file with dynamic file name
dir = Path("W:/019_Glassdoor/1 Data/1 Glassdoor Links/MnA Sample Eikon")
path_with_time = Path.joinpath(dir, f'{time.strftime("%m%d")}_MnA_Sample_Eikon_unfiltered.csv')

df_ma.to_csv(path_with_time, sep=";", decimal=",", float_format='%.2f', index=False) #english decimal

print(f"{path_with_time} saved")

W:\019_Glassdoor\1 Data\1 Glassdoor Links\MnA Sample Eikon\0920_MnA_Sample_Eikon_unfiltered.csv saved


### prepare sample

In [107]:
df = df_ma.copy()

df = df.rename(columns={"Instrument": "eikon_deal_id", "Total Assets Last 12 Months": "Target Total Assets Last 12 Months"}) #rename
df['Date Announced'] = pd.to_datetime(df['Date Announced']) #convert date strings to date objects

# filter df
condition = (
    (df["Target Public Status"] == "Public") #exclude subsidiary
    & (df["Acquiror Public Status"] == "Public") 
    & (~df['Date Announced'].dt.year.isin([])) #exclude 2008, 2009, 2010
    # & (df["Target PermID"].isna() == True) #exclude empty PermID //some of them have financials -> keep them
    # & (df["Target 6-digit CUSIP"].isna() == True) # none missing
    # & (df['Total Assets Last 12 Months'] < df['Total Assets Last 12 Months'].quantile(0.1)) # < : exclude all above quantile
    )
df = df[condition]

df.head()
df.shape

,eikon_deal_id,SDC Deal No,Date Announced,Date Effective or Unconditional,Date Effective,Deal Status,Target Full Name,Target PermID,Target 6-digit CUSIP,Target Nation,...,Acquiror Full Name,Acquiror PermID,Acquiror 6-digit CUSIP,Acquiror's SEDOL,Acquiror Nation,Acquiror Public Status,Acquiror Total Assets Last 12 Months,Deal Value,Percentage of Shares Held by Acquiror 6 Months Prior to Announcement,Percentage of Shares Owned after Transaction
3,34391281082,1794406020,2008-09-15,2008-10-30,2008-10-30,Completed,Napster Inc,4295899555.0,630797,United States,...,Best Buy Co Inc,4295903556.0,086516,2094670,United States,Public,17522000000.0,126.935,0.0,90.191
4,34391290134,1879131020,2008-04-24,2008-09-29,2008-09-29,Completed,Wendy's International Inc,4295905330.0,950590,United States,...,Triarc Cos Inc,4295905130.0,895927,B3NXMJ9,United States,Public,1390735000.0,2345.517,0.0,100.0
8,34391292476,1900306020,2008-01-28,2008-08-22,2008-08-22,Completed,NYMEX Holdings Inc,8589934166.0,62948N,United States,...,CME Group Inc,4295899615.0,12572Q,2965839,United States,Public,18562632000.0,7555.372,0.0,100.0
14,34391297274,1941059020,2008-01-07,2008-04-01,2008-04-01,Completed,Burke-Parsons-Bowlby Corp,4296470998.0,121368,United States,...,Stella-Jones Inc,5048817617.0,85853F,2809777,Canada,Public,263939000.0,76.132,0.0,100.0
17,34391297298,1941224020,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",4295907031.0,55272D,United States,...,MutualFirst Financial Inc,4295914259.0,62845B,2620343,United States,Public,957343000.0,57.168,0.0,100.0


(3144, 22)

### cusip to ISIN (totally unreliable, use permid to isin instead)

In [133]:
firms_cusips = df["Target 6-digit CUSIP"].tolist()

df_cusips = ek.get_symbology(firms_cusips, from_symbol_type='CUSIP', to_symbol_type=None, raw_output=False, debug=False)
# to_symbol_type=None -> all are requested. CUSIP , ISIN , SEDOL , RIC , ticker , lipperID , IMO , and OAPermID

df_cusips = df_cusips[df_cusips["ISIN"].isna() == False]
df_cusips

,SEDOL,OAPermID,CUSIP,ISIN,ticker,error,RIC,lipperID
62948N,<NA>,8589934166,62948N104,US62948N1046,<NA>,<NA>,<NA>,<NA>
55272D,<NA>,4295907031,55272D106,US55272D1063,<NA>,<NA>,<NA>,<NA>
82511E,<NA>,4295907912,82511E109,US82511E1091,<NA>,<NA>,<NA>,<NA>
02342J,<NA>,4295910045,02342J101,US02342J1016,<NA>,<NA>,<NA>,<NA>
92839Y,<NA>,4295899729,92839Y109,US92839Y1091,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...
02301T,<NA>,4296352310,02301T108,CA02301T1084,<NA>,<NA>,<NA>,<NA>
83416M,<NA>,5001234308,83416M105,US83416M1053,<NA>,<NA>,<NA>,<NA>
87402X,<NA>,5063731294,87402X108,CA87402X1087,TGC,<NA>,<NA>,<NA>
1J0099,<NA>,<NA>,3128LUDC7,US3128LUDC78,<NA>,<NA>,<NA>,<NA>


### permid to isin

#### target

In [138]:
target_cusips = df["Target PermID"].astype("Int64").astype(str).tolist() #first convert to int (without decimal), then to str, since get_data needs strings in the list
fields = ['TR.ISIN', 'TR.CUSIP', 'TR.RIC'] # check the data item browser
params = {'Scale': 6, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'EUR'} # set EDate to minus values for the most recent years

df_isin_target_retrieve, err = ek.get_data(target_cusips, fields, params)

df_isin_target_retrieve

,Instrument,ISIN,CUSIP,RIC
0,4295899555,US6307971084,630797108,NAPS.OQ^K08
1,4295905330,US9505901093,950590109,WEN.N^I08
2,8589934166,US62948N1046,62948N104,NMX.N^H08
3,4296470998,US1213681042,121368104,BPAB.PK^E08
4,4295907031,US55272D1063,55272D106,MFBC.OQ^G08
...,...,...,...,...
3139,5064695004,CA72941N1006,72941N100,PLUS.CD^I21
3140,5000095185,AU000000WIC6,,WIC.AX
3141,4295856861,AU000000OZG8,,OZG.AX
3142,5079547980,GB00BMDNH979,,OCDX.OQ


In [139]:
df_isin_target = df_isin_target_retrieve.copy()
#replace empty strings with np nan
for columnname in ['ISIN', 'CUSIP', 'RIC']:
  df_isin_target[columnname] = df_isin_target[columnname].replace({"":np.nan})

df_isin_target = df_isin_target.rename(columns={"Instrument": "Target PermID"})

# add prefix to columns (except first column)
new_names = [(i,'Target_'+i) for i in df_isin_target.iloc[:, 1:].columns.values] #set up dict entries for each columns name {"old": "Target_old"}
df_isin_target.rename(columns = dict(new_names), inplace=True)

df_isin_target = df_isin_target[df_isin_target["Target_ISIN"].isna() == False]
df_isin_target

,Target PermID,Target_ISIN,Target_CUSIP,Target_RIC
0,4295899555,US6307971084,630797108,NAPS.OQ^K08
1,4295905330,US9505901093,950590109,WEN.N^I08
2,8589934166,US62948N1046,62948N104,NMX.N^H08
3,4296470998,US1213681042,121368104,BPAB.PK^E08
4,4295907031,US55272D1063,55272D106,MFBC.OQ^G08
...,...,...,...,...
3139,5064695004,CA72941N1006,72941N100,PLUS.CD^I21
3140,5000095185,AU000000WIC6,<NA>,WIC.AX
3141,4295856861,AU000000OZG8,<NA>,OZG.AX
3142,5079547980,GB00BMDNH979,<NA>,OCDX.OQ


#### acq

In [141]:
acquiror_cusips = df["Acquiror PermID"].astype("Int64").astype(str).tolist() #first convert to int (without decimal), then to str, since get_data needs strings in the list
fields = ['TR.ISIN', 'TR.CUSIP', 'TR.RIC'] # check the data item browser
params = {'Scale': 6, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'EUR'} # set EDate to minus values for the most recent years

df_isin_acquiror_retrieve, err = ek.get_data(acquiror_cusips, fields, params)


df_isin_acquiror = df_isin_acquiror_retrieve.copy()
#replace empty strings with np nan
for columnname in ['ISIN', 'CUSIP', 'RIC']:
  df_isin_acquiror[columnname] = df_isin_acquiror[columnname].replace({"":np.nan})

df_isin_acquiror = df_isin_acquiror.rename(columns={"Instrument": "Acquiror PermID"})

# add prefix to columns (except first column)
new_names = [(i,'Acquiror_'+i) for i in df_isin_acquiror.iloc[:, 1:].columns.values] #set up dict entries for each columns name {"old": "acquiror_old"}
df_isin_acquiror.rename(columns = dict(new_names), inplace=True)

df_isin_acquiror = df_isin_acquiror[df_isin_acquiror["Acquiror_ISIN"].isna() == False]
df_isin_acquiror

,Acquiror PermID,Acquiror_ISIN,Acquiror_CUSIP,Acquiror_RIC
0,4295903556,US0865161014,086516101,BBY.N
1,4295905130,US95058W1009,95058W100,WEN.OQ
2,4295899615,US12572Q1058,12572Q105,CME.OQ
3,5048817617,CA85853F1053,85853F105,SJ.TO
4,4295914259,US62845B1044,62845B104,MFSF.OQ^D20
...,...,...,...,...
3139,5068929693,CA3771304068,377130406,GLASau.NLB
3140,4295858256,AU000000WAM2,<NA>,WAM.AX
3141,4295858256,AU000000WAM2,<NA>,WAM.AX
3142,4295907711,US74838J1016,74838J101,QDEL.OQ


In [148]:
for columnname in ["Acquiror PermID", "Target PermID"]: #convert to int, since isin dfs use int for the permid
    df[columnname] = df[columnname].astype("Int64")

df_final = df.merge(df_isin_acquiror.iloc[:,:2], how="left", on="Acquiror PermID")
df_final = df.merge(df_isin_target.iloc[:,:2], how="left", on="Target PermID")
df_final

,eikon_deal_id,SDC Deal No,Date Announced,Date Effective or Unconditional,Date Effective,Deal Status,Target Full Name,Target PermID,Target 6-digit CUSIP,Target Nation,...,Acquiror PermID,Acquiror 6-digit CUSIP,Acquiror's SEDOL,Acquiror Nation,Acquiror Public Status,Acquiror Total Assets Last 12 Months,Deal Value,Percentage of Shares Held by Acquiror 6 Months Prior to Announcement,Percentage of Shares Owned after Transaction,Target_ISIN
0,34391281082,1794406020,2008-09-15,2008-10-30,2008-10-30,Completed,Napster Inc,4295899555,630797,United States,...,4295903556,086516,2094670,United States,Public,17522000000.0,126.935,0.0,90.191,<NA>
1,34391290134,1879131020,2008-04-24,2008-09-29,2008-09-29,Completed,Wendy's International Inc,4295905330,950590,United States,...,4295905130,895927,B3NXMJ9,United States,Public,1390735000.0,2345.517,0.0,100.0,<NA>
2,34391292476,1900306020,2008-01-28,2008-08-22,2008-08-22,Completed,NYMEX Holdings Inc,8589934166,62948N,United States,...,4295899615,12572Q,2965839,United States,Public,18562632000.0,7555.372,0.0,100.0,<NA>
3,34391297274,1941059020,2008-01-07,2008-04-01,2008-04-01,Completed,Burke-Parsons-Bowlby Corp,4296470998,121368,United States,...,5048817617,85853F,2809777,Canada,Public,263939000.0,76.132,0.0,100.0,<NA>
4,34391297298,1941224020,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",4295907031,55272D,United States,...,4295914259,62845B,2620343,United States,Public,957343000.0,57.168,0.0,100.0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,154087420083,3850353040,2021-12-20,2022-04-28,2022-04-28,Completed,Plus Products Inc,5064695004,6H4512,Canada,...,5068929693,377130,BMVQ9T2,United States,Public,258564000.0,25.6,0.0,100.0,<NA>
3140,154087421587,3851037040,2021-12-23,2022-04-21,2022-04-21,Completed,Westoz Investment Co Ltd,5000095185,96170V,Australia,...,4295858256,93362V,6164988,Australia,Public,1746310000.0,156.801224,0.0,100.0,<NA>
3141,154087421802,3851153040,2021-12-23,2022-04-21,2022-04-21,Completed,Ozgrowth Ltd,4295856861,69699Q,Australia,...,4295858256,93362V,6164988,Australia,Public,1746310000.0,106.059947,0.0,100.0,<NA>
3142,154087421847,3851185020,2021-12-23,2022-05-27,2022-05-27,Completed,Ortho Clinical Diagnostics Holdings PLC,5079547980,G6829J,United States,...,4295907711,74838J,<NA>,United States,Public,2062388000.0,8094.559,0.0,100.0,<NA>


In [6]:
## save file with dynamic file name
dir = Path("W:/019_Glassdoor/1 Data/1 Glassdoor Links/MnA Sample Eikon")
path_with_time = Path.joinpath(dir, f'{time.strftime("%m%d")}_MnA_Sample_Eikon.csv')

df_ma.to_csv(path_with_time, sep=";", decimal=",", float_format='%.2f', index=False) #english decimal

print(f"{path_with_time} saved")

W:\019_Glassdoor\1 Data\1 Glassdoor Links\MnA Sample Eikon\0920_MnA_Sample_Eikon.csv saved


## get panel data

### firm fundamentals

In [ ]:
firms = ['GOOG.O', 'MSFT.O', 'FB.O', 'AMZN.O', 'TWTR.K'] # check the screener
fields = ['TR.Revenue.date','TR.Revenue','TR.GrossProfit', 'TR.ISIN'] # check the data item browser
params = {'Scale': 6, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'EUR'} # set EDate to minus values for the most recent years

df, err = ek.get_data(firms, 
                      fields,
                      params)

df

,Instrument,Date,Revenue,Gross Profit,ISIN
0,GOOG.O,2021-12-31T00:00:00Z,226632.96342,129044.36268,US02079K1079
1,MSFT.O,2022-06-30T00:00:00Z,189153.5454,129384.1924,US5949181045
2,FB.O,2021-12-31T00:00:00Z,103737.42414,83814.0048,US30303M1027
3,AMZN.O,2021-12-31T00:00:00Z,413283.62052,173713.49748,US0231351067
4,TWTR.K,2021-12-31T00:00:00Z,4466.457816,2885.26017,US90184L1026


### symbology converter

In [ ]:
firms = ['GOOG.O', 'MSFT.O', 'FB.O', 'AMZN.O', 'TWTR.K'] # check the screener
fields = ['TR.RIC', 'TR.CUSIP', 'TR.SEDOL', 'TR.ISIN', 'TR.OrganizationID'] # check the data item browser
params = {'Scale': 6, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'EUR'} # set EDate to minus values for the most recent years

df, err = ek.get_data(firms, 
                      fields,
                      params)

df

,Instrument,RIC,CUSIP,SEDOL,ISIN,Organization PermID
0,GOOG.O,GOOG.O,02079K107,BYY88Y7,US02079K1079,5030853586
1,MSFT.O,MSFT.O,594918104,2588173,US5949181045,4295907168
2,FB.O,FB.O,30303M102,B7TL820,US30303M1027,4297297477
3,AMZN.O,AMZN.O,023135106,2000019,US0231351067,4295905494
4,TWTR.K,TWTR.K,90184L102,BFLR866,US90184L1026,4296301199


In [ ]:
firm_ids = df["RIC"].tolist()
df_symb = ek.get_symbology(firm_ids, from_symbol_type='RIC', to_symbol_type=None, raw_output=False, debug=False)
# to_symbol_type=None -> all are requested. CUSIP , ISIN , SEDOL , RIC , ticker , lipperID , IMO , and OAPermID

df_symb

,CUSIP,ISIN,OAPermID,RIC,ticker
GOOG.O,02079K107,US02079K1079,5030853586,GOOG.O,GOOG
MSFT.O,594918104,US5949181045,4295907168,MSFT.O,MSFT
FB.O,30303M102,US30303M1027,4297297477,META.O,META
AMZN.O,023135106,US0231351067,4295905494,AMZN.O,AMZN
TWTR.K,90184L102,US90184L1026,4296301199,TWTR.K,TWTR


In [76]:
df_permToIsin, err = ek.get_data(["4295907168", "CA51818R2046"], "TR.ISIN, TR.EBIT")
df_permToIsin

,Instrument,ISIN,EBIT
0,4295907168,US5949181045,83383000000.0
1,CA51818R2046,CA51818R2046,<NA>
